In [3]:
import os
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
import requests
import chromadb
from paddleocr import PaddleOCR
import time

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


In [4]:
!pip install langchain



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [10]:
#creating chunks
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_combined_text(file_path, chunk_size=500, chunk_overlap=50):
    with open(file_path, "r", encoding="utf-8") as f:
        full_text = f.read()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    chunks = text_splitter.split_text(full_text)
    print(f"Generated {len(chunks)} chunks.")
    return chunks

combined_file = "extracted_text/combined_summary.txt"
chunks = chunk_combined_text(combined_file)

#chunks file
os.makedirs("chunked_output", exist_ok=True)
for idx, chunk in enumerate(chunks):
    with open(f"chunked_output/chunk_{idx + 1}.txt", "w", encoding="utf-8") as f:
        f.write(chunk)


Generated 129 chunks.


In [11]:
#Creating Embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('BAAI/bge-base-en-v1.5')

def generate_embeddings(chunks):
    embeddings = model.encode(chunks, show_progress_bar=True).tolist()
    print(f" Generated {len(embeddings)} embeddings.")
    return embeddings

embeddings = generate_embeddings(chunks)



Batches: 100%|██████████| 5/5 [00:17<00:00,  3.56s/it]

 Generated 129 embeddings.


In [12]:
import chromadb
from chromadb.config import Settings

#chroma db
client = chromadb.Client(Settings(persist_directory="./chroma_store"))  
collection = client.get_or_create_collection("document_chunks")

def store_in_chromadb(chunks, embeddings):
    for idx, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        collection.add(
            documents=[chunk],
            embeddings=[embedding],
            ids=[str(idx)]
        )
    print("All chunks stored in ChromaDB.")


def query_chromadb(query, top_k=3):
    query_embedding = model.encode([query]).tolist()[0]
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return results["documents"][0]


store_in_chromadb(chunks, embeddings)



All chunks stored in ChromaDB.


In [5]:
question = "What is the company's free cash flow trend?"
top_chunks = query_chromadb(question)
print("\nTop relevant chunk(s):\n", "\n---\n".join(top_chunks))


Top relevant chunk(s):
 REVENUE PROFITABILITY OTHER
Revenue from Operations EBITDA EBITDA Margin Free Cash Flow”
19.8 Cr. 0.02 Cr. 0.1% 0.6 Cr.
& 8% YoY 0.17 Cr. YoY, down Margin reduced by 0.9% 0.9 Cr. YoY, down
from Q1 FY23 from Q1 FY23 from Q1 FY23
Total Income PAT PAT Margin Cash & Cash Equivalents
20.8 Cr. 0.7 Cr. 3.5 % 61.9 Cr.
A 9% YoY 0.1 Cr. YoY, down Margin reduced by 1.1% A 30% YoY
from Q1 FY23 from Q1 FY23 & 14.1 Cr. YoY

In INR

q yTracxn

--- page_6.txt ---
Q1 FY24: Continued Revenue Growth Momentum
---
Cash & Cash Equivalents'*

FCF (Free Cash Flow)' (0.9)Cr.
(in INR Cr.) (in INR Cr.)
Historical Annual i 61.9 Cr.
a Added 14.1 Cr. YoY at41cr.
me & 30% YoY A 30%
0.6
61.9

60.3

49
45.8 478
30.4

QIFY23. QIFY24

(6.1)
FY 21 FY 22 FY 23
Revenue increased by 8%, collections increased but FCF FY 21

lower due to increased employee, rent and other expense
@Tracxn

FY 22 FY 23 QIFY23 © QIFY24

--- page_21.txt ---
Q1 FY24: Other KPls (1/2)
---
expenses, reimbursable to the compa

In [13]:
#Lanchchain Model
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    openai_api_key="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",  
    model_name="llama3-8b-8192" 
)

def retrieve_and_answer(question, top_k=3):
    top_chunks = query_chromadb(question, top_k=top_k)
    context = "\n\n".join(top_chunks)
    prompt = f"""You are an assistant answering strictly based on the provided context.

Context:
{context}

Question:
{question}

Answer:"""
    response = llm.predict(prompt)
    return response


#ask questions
question = "\n what is revenue from operations "
answer = retrieve_and_answer(question, top_k=3)
print("\nAnswer:", answer)




Answer: According to the provided context, the Revenue from Operations for Q1 FY24 is INR 19.82 Cr.


NameError: name 'top_chunks' is not defined

In [ ]:
!pip install openai